In [1]:
import numpy as np
from tqdm import tqdm
from solver import qp
import pandas as pd
import gzip
import pickle
import warnings
from generate_instances import generate_softmarginsvm, generate_markowitz_portfolio_optimization, generate_convex_function_to_data, generate_regressor_selection

### Soft margin Support Vector Machine

In [2]:
def surrogate_gen():
    NUM = np.random.randint(40, 51) # Number of points
    DIM = 2 # Dimension of the points
    C = 10 # regularization parameter (hardness of the margin)
    
    # 2-D mean of ones
    M1 = np.ones((DIM,))
    # 2-D mean of threes
    M2 = 3 * np.ones((DIM,))
    # 2-D covariance of 0.3
    C1 = np.diag(0.3 * np.ones((DIM,)))
    # 2-D covariance of 0.2
    C2 = np.diag(0.2 * np.ones((DIM,)))

    # generate NUM points from gaussian 1
    x1 = np.random.multivariate_normal(M1, C1, NUM) #generate gaussian
    # labels
    y1 = np.ones((x1.shape[0],))
    # generate NUM points from gaussian 2
    x2 = np.random.multivariate_normal(M2, C2, NUM)
    y2 = -np.ones((x2.shape[0],))
    # join
    X = np.concatenate((x1, x2), axis = 0)
    y = np.concatenate((y1, y2), axis = 0)

    Q, q, G, h, A, b, S = generate_softmarginsvm(y, X, C)    
    return Q, q, G, h, A, b, S

### Markowitz portfolio optimization

In [4]:
def surrogate_gen():
    n_obs = 252
    n_assets = np.random.randint(45, 56)
    artificial_returns = np.random.randn(n_obs, n_assets) + 0.05 # minimum expected return
    returns = pd.DataFrame(artificial_returns)
    pbar = np.array(returns.mean()).reshape(1, -1)
    r_min = pbar.mean() # minimum return threshold
    Q, q, G, h, A, b, S = generate_markowitz_portfolio_optimization(artificial_returns, r_min, pbar)
    return Q, q, G, h, A, b, S

### Fitting convex function to given data

In [6]:
def surrogate_gen():
    
    # Generate 'u' values (from -2 to 2) for the independent variable
    NUM = np.random.randint(20, 26) # Number of points
    u_values = np.linspace(-2.0, 2.0, num=NUM).reshape(-1, 1)
    Q, q, G, h, A, b, S = generate_convex_function_to_data(u_values)
    return Q, q, G, h, A, b, S

### Regressor selection problem

In [8]:
def surrogate_gen():
    
    # Set dimensions
    m = 30  # number of rows for A
    n = 60  # number of columns for A
    nopts = 100 # range of alpha values
    alpha_index = np.random.randint(0, nopts-1) # Select an alpha value from the range nopts
    Q, q, G, h, A, b, S = generate_regressor_selection(alpha_index, m, n)
    
    return Q, q, G, h, A, b, S

### create

In [ ]:
import os

# directory = f'../../../../work/log1/darius.weber/Quadratic_Programming_Datasets/raw'
directory = f'Quadratic_Programming_Datasets/raw'
os.makedirs(directory, exist_ok=True)
warnings.filterwarnings("error")
qp.options['show_progress'] = False
ips = []
success_cnt = 0
fail_cnt = 0
max_ipm_steps = 0
pkg_idx = 0
num = 400
max_iter = 2*num


for i in tqdm(range(max_iter)):
    Q, q, G, h, A, b, S = surrogate_gen()
    try:
        res = qp.qp(Q, q, G, h, A, b, callback=lambda res: res)
        max_ipm_steps = max(max_ipm_steps, res['iterations'])
    except Exception as e:
        fail_cnt += 1
        warnings.warn(f'Optimization failed with error: {str(e)}')
        continue
    else:
        if res['status'] == 'optimal': 
            ips.append((Q, q, G, h, A, b, S))
            success_cnt += 1
            
    if len(ips) >= 1000 or success_cnt == num:
        ips = [ip + (max_ipm_steps,) for ip in ips]
        print("max ipm steps", max_ipm_steps)
        with gzip.open(f'{directory}/instance_{pkg_idx}.pkl.gz', "wb") as file:
            pickle.dump(ips, file)
            print(f'Package {pkg_idx} saved')
            pkg_idx += 1

        ips = []
    if success_cnt >= num:
        break
warnings.resetwarnings()

In [ ]:
import os
# Ensure the output directory exists
os.makedirs('output', exist_ok=True)
number = 0
# Open and load the .pkl.gz file
with gzip.open(f'Quadratic_Programming_Datasets/raw/instance_{number}.pkl.gz', 'rb') as f:
    data = pickle.load(f)

# Now `data` contains the contents of the pickle file
output_path = f'output/output{number}.txt'
with open(output_path, 'w') as output_file:
    output_file.write(str(data))
print(f"Data has been written to output/output{number}.txt")